# PhotonLibrary + MLP
**This notebook: [see on github](https://github.com/drinkingkazu/2019-06-17-Notebooks/blob/master/MLP%20for%20PhotonLibrary.ipynb) or [run it on google colab](https://colab.research.google.com/github/drinkingkazu/2019-06-17-Notebooks/blob/master/MLP%20for%20PhotonLibrary.ipynb)**.

A photon library is a look-up map for a LArTPC detector. Provided (x,y,z) position, a photon library provides a probability of detecting a photon by individual PMT. Why do we use a photon library? Because simulation of individual photon propagation requires ray tracing, and it is hard to use a parametric function for approximation due to stochastic physics processes (scattering) and often a complicated detector geometry (field cage, anode wires, cryostat, etc.). A downside of using a photon library is the memory as it needs to hold infomration of probability per PMT (floating point value * PMT counts) for every single voxelized volume in the detector.

An alternative method to a photon library is a neural network, which is good at function approximation. This notebook is an attempt to emulate a photon library with MLP.

In [1]:
from __future__ import print_function
import matplotlib
%matplotlib inline
import torch
import numpy as np
SEED=123
_=np.random.seed(SEED)
_=torch.manual_seed(SEED)

! [ -d kworkshop ] || git clone -b 2019-06-17-NeuralNets https://github.com/drinkingkazu/kworkshop
! cd kworkshop && git pull && cd -

Already up to date.
/content


Instantiate a photon library

In [0]:
from kworkshop.utils import PhotonLibrary
plib=PhotonLibrary()

In [3]:
class dataset:
  
  def __init__(self,plib,num_points=10000):
    self._plib = plib
    self._num_points = num_points
    
  def __len__(self):
    return 100000
  
  def __getitem__(self,index):
    return self.random_points(self._num_points)
  
  def random_points(self, num_points):
  
    points = np.zeros(shape=(num_points,3), dtype=np.float32)
    ranges = (self._plib.XRange(), self._plib.YRange(), self._plib.ZRange())

    for index, r in enumerate(ranges):
      points[:,index] = r[0] + (r[1]-r[0]) * np.random.ranf(num_points)

    voxel_ids = [ self._plib.GetVoxelID(pt[0], pt[1], pt[2]) for pt in points ]
    for i, vox in enumerate(voxel_ids):
      if vox > self._plib._plib.shape[1]:
        print(vox,points[i][0],points[i][1],points[i][2])

    visibility = np.array([ [ self._plib._plib[pmt][voxel] for pmt in range(180) ] for voxel in voxel_ids ])
    visibility = visibility.astype(np.float32)

    return points, visibility
  
d = dataset(plib)
d.random_points(100)[0].shape


(100, 3)

In [0]:
from torch.utils.data import DataLoader

plib_dataset = dataset(plib,num_points=1000)
loader       = DataLoader(plib_dataset, batch_size=4, num_workers=16)

In [5]:
import time

t0=time.time()
for index, data in enumerate(loader):
  t1=time.time()
  print('Batch',index,'Time',t1-t0)
  t0=time.time()
  
  if index==39: break


Batch 0 Time 8.188227653503418
Batch 1 Time 0.0003170967102050781
Batch 2 Time 0.6615324020385742
Batch 3 Time 0.00037384033203125
Batch 4 Time 0.0003077983856201172
Batch 5 Time 0.0003082752227783203
Batch 6 Time 0.02953195571899414
Batch 7 Time 0.09671759605407715
Batch 8 Time 0.0003685951232910156
Batch 9 Time 0.0003478527069091797
Batch 10 Time 0.0003204345703125
Batch 11 Time 0.0003249645233154297
Batch 12 Time 0.00034999847412109375
Batch 13 Time 0.0003459453582763672
Batch 14 Time 0.01885390281677246
Batch 15 Time 0.0003266334533691406
Batch 16 Time 5.00968337059021
Batch 17 Time 0.6126365661621094
Batch 18 Time 0.0003592967987060547
Batch 19 Time 0.12052559852600098
Batch 20 Time 0.00038170814514160156
Batch 21 Time 0.0003192424774169922
Batch 22 Time 0.00035834312438964844
Batch 23 Time 0.00038814544677734375
Batch 24 Time 0.14615488052368164
Batch 25 Time 0.0003905296325683594
Batch 26 Time 0.0003135204315185547
Batch 27 Time 0.00036978721618652344
Batch 28 Time 0.00033020973

In [0]:
class MLP(torch.nn.Module):
    def __init__(self):
        
        super(MLP, self).__init__()        
        # MLP w/ 2 hidden layers, 1024 neurons each
        self._estimator = torch.nn.Sequential(
            torch.nn.Linear(3,   1024), torch.nn.ReLU(),
            torch.nn.Linear(1024,1024), torch.nn.ReLU(),
            torch.nn.Linear(1024,180),  torch.nn.ReLU()
        )

    def forward(self, x):
        return self._estimator(x)

In [0]:
class BLOB:
    pass
blob=BLOB()
blob.net       = MLP().cuda() # construct Lenet, use GPU
blob.criterion = torch.nn.SmoothL1Loss() # use Huber loss to define an error
blob.optimizer = torch.optim.Adam(blob.net.parameters()) # use Adam optimizer algorithm
blob.iteration = 0    # integer count for the number of train steps
blob.data      = None # data for training/analysis
blob.label     = None # label for training/analysis

In [0]:
def forward(blob,train=True):
    """
       Args: blob should have attributes, net, criterion, softmax, data, label
       
       Returns: a dictionary of predicted labels, softmax, loss, and accuracy
    """
    with torch.set_grad_enabled(train):
        # Prediction
        data = blob.data.cuda()
        prediction = blob.net(data)
        # Training
        loss,acc=-1,-1
        if blob.label is not None:
            label = blob.label.cuda()
            loss = blob.criterion(prediction,label)
            
        blob.loss = loss
        
        prediction = prediction.cpu().detach().numpy()
        
        return {'prediction' : prediction,
                'loss'       : loss.cpu().detach().item()}
      
def backward(blob):
    blob.optimizer.zero_grad()  # Reset gradients accumulation
    blob.loss.backward()
    blob.optimizer.step()

In [0]:
def train_loop(blob,train_loader,num_iteration):
    # Set the network to training mode
    blob.net.train()
    # Let's record the loss at each iteration and return
    train_loss=[]
    # Loop over data samples and into the network forward function
    for i,data in enumerate(train_loader):
        if blob.iteration >= num_iteration:
            break
        blob.iteration += 1
        # data and label
        blob.data, blob.label = data
        # call forward
        res = forward(blob,True)
        # Recird loss
        train_loss.append(res['loss'])
        # once in a while, report
        if blob.iteration == 0 or (blob.iteration+1)%10 == 0:
            print('Iteration',blob.iteration,'... Loss',res['loss'])
        backward(blob)
    return np.array(train_loss)

In [10]:
loss = train_loop(blob,loader,100)

Iteration 9 ... Loss 3.2338798519049305e-06
Iteration 19 ... Loss 5.071385658084182e-06
Iteration 29 ... Loss 5.071385658084182e-06
Iteration 39 ... Loss 4.1226480789191555e-06
Iteration 49 ... Loss 5.713039172405843e-06
Iteration 59 ... Loss 5.713039172405843e-06
Iteration 69 ... Loss 5.469317784445593e-06
Iteration 79 ... Loss 5.469317784445593e-06
Iteration 89 ... Loss 3.724443331520888e-06
Iteration 99 ... Loss 5.1737119974859525e-06


In [11]:
fig,ax=plt.subplots(figsize=(12,8),facecolor='w')
plt.plot(range(len(loss)),loss,marker="",linewidth=2,color='blue')
from kworkshop.utils import moving_average 
plt.plot(moving_average(range(len(loss)),30),moving_average(loss,30),marker="",linewidth=2,color='red')
plt.show()

NameError: ignored